<a href="https://colab.research.google.com/github/sfirdous/MachineLearning/blob/main/ML2/Lab/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.datasets import load_iris
import torch

In [6]:
class KNearestNeighbour():
  def __init__(self,X,y):
    self.__N = X.shape[0]         # number of samples
    self.__d = X.shape[1]         # number of features


  def train_test_split(self,X,y,split_ratio):
    # select random indices
    indices = torch.randperm(self.__N)

    train_N = int(split_ratio * self.__N)

    train_indices = indices[:train_N]
    test_indices = indices[train_N:]

    X_train = X[train_indices]
    y_train = y[train_indices]

    X_test = X[test_indices]
    y_test = X[test_indices]



